In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [24]:
url_inicial = 'http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/archive/3'
url_root = 'http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/archive/3'
r=requests.get(url_inicial)

In [25]:
soup = BeautifulSoup(r.text, 'html.parser')

In [26]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/view/55',
 'http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/view/52',
 'http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/view/51',
 'http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/view/256',
 'http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/view/270',
 'http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/view/271',
 'http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/view/272']

In [27]:
def get_url_items(sopa, url): 
    box = soup.find('ul', class_='issues_archive')
    volumen=box.findAll('div', class_='obj_issue_summary')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [28]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina http://www.revistalaventana.cucsh.udg.mx/index.php/LV/issue/archive/3


In [29]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

7

In [30]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [31]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista de Estudios de Género, la Ventana'
    a='V. Ciencias Sociales'
    tem='Estudios de Género'
    url='http://www.revistalaventana.cucsh.udg.mx'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h1').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('div', class_='description').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [32]:
list_scraper=list_scraper[0:25]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6


In [33]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,"Revista de Estudios de Género, la Ventana",V. Ciencias Sociales,Estudios de Género,Vol. 1 Núm. 7 (1998),None,http://www.revistalaventana.cucsh.udg.mx,http://www.revistalaventana.cucsh.udg.mx/index...
1,"Revista de Estudios de Género, la Ventana",V. Ciencias Sociales,Estudios de Género,Vol. 1 Núm. 6 (1997),None,http://www.revistalaventana.cucsh.udg.mx,http://www.revistalaventana.cucsh.udg.mx/index...
2,"Revista de Estudios de Género, la Ventana",V. Ciencias Sociales,Estudios de Género,Núm. 5 (1997),None,http://www.revistalaventana.cucsh.udg.mx,http://www.revistalaventana.cucsh.udg.mx/index...
3,"Revista de Estudios de Género, la Ventana",V. Ciencias Sociales,Estudios de Género,Núm. 4 (1996),None,http://www.revistalaventana.cucsh.udg.mx,http://www.revistalaventana.cucsh.udg.mx/index...
4,"Revista de Estudios de Género, la Ventana",V. Ciencias Sociales,Estudios de Género,Núm. 3 (1996),None,http://www.revistalaventana.cucsh.udg.mx,http://www.revistalaventana.cucsh.udg.mx/index...
5,"Revista de Estudios de Género, la Ventana",V. Ciencias Sociales,Estudios de Género,Núm. 2 (1995),None,http://www.revistalaventana.cucsh.udg.mx,http://www.revistalaventana.cucsh.udg.mx/index...
6,"Revista de Estudios de Género, la Ventana",V. Ciencias Sociales,Estudios de Género,Núm. 1 (1995),None,http://www.revistalaventana.cucsh.udg.mx,http://www.revistalaventana.cucsh.udg.mx/index...


In [34]:
df_catalogo.to_csv('Revista171.03.csv', index=False)